In [1]:
!wget http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/balanced_train_segments.csv

--2021-12-06 15:39:17--  http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/balanced_train_segments.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.24.112, 172.217.24.240, 172.217.25.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.24.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1211931 (1.2M) [application/octet-stream]
Saving to: ‘balanced_train_segments.csv’

balanced_train_segm 100%[===================>]   1.16M  2.88MB/s    in 0.4s    

2021-12-06 15:39:18 (2.88 MB/s) - ‘balanced_train_segments.csv’ saved [1211931/1211931]



In [1]:
import pandas as pd

df_dirty = pd.read_csv("balanced_train_segments.csv", delimiter=' ', header=2)
df = df_dirty.copy()
for i in range(1, len(df.columns)):
  df[df.columns[i]] = df_dirty[df.columns[i-1]]
df = df.drop(columns='#')

In [2]:
df

,"YTID,","start_seconds,","end_seconds,",positive_labels
0,"--PJHxphWEs,","30.000,","40.000,","/m/09x0r,/t/dd00088"
1,"--ZhevVpy1s,","50.000,","60.000,",/m/012xff
2,"--aE2O5G5WE,","0.000,","10.000,","/m/03fwl,/m/04rlf,/m/09x0r"
3,"--aO5cdqSAg,","30.000,","40.000,","/t/dd00003,/t/dd00005"
4,"--aaILOrkII,","200.000,","210.000,","/m/032s66,/m/073cg4"
...,...,...,...,...
22155,"zyqg4pYEioQ,","20.000,","30.000,","/m/09x0r,/m/0llzx"
22156,"zz0ddNfz0h0,","30.000,","40.000,","/m/012f08,/m/03cl9h,/m/07yv9,/m/0k4j"
22157,"zz8TGV83nkE,","80.000,","90.000,","/m/012f08,/m/02mk9,/m/04_sv,/m/07yv9"
22158,"zzlK8KDqlr0,","370.000,","380.000,","/m/01m2v,/m/07qc9xj,/m/09x0r,/t/dd00125"


## 1. Crawling the Dataset
- using [`yt-dlp`](https://github.com/yt-dlp/yt-dlp)

In [60]:
!pip install -Uq yt-dlp

In [3]:
import os

In [4]:
df

,"YTID,","start_seconds,","end_seconds,",positive_labels
0,"--PJHxphWEs,","30.000,","40.000,","/m/09x0r,/t/dd00088"
1,"--ZhevVpy1s,","50.000,","60.000,",/m/012xff
2,"--aE2O5G5WE,","0.000,","10.000,","/m/03fwl,/m/04rlf,/m/09x0r"
3,"--aO5cdqSAg,","30.000,","40.000,","/t/dd00003,/t/dd00005"
4,"--aaILOrkII,","200.000,","210.000,","/m/032s66,/m/073cg4"
...,...,...,...,...
22155,"zyqg4pYEioQ,","20.000,","30.000,","/m/09x0r,/m/0llzx"
22156,"zz0ddNfz0h0,","30.000,","40.000,","/m/012f08,/m/03cl9h,/m/07yv9,/m/0k4j"
22157,"zz8TGV83nkE,","80.000,","90.000,","/m/012f08,/m/02mk9,/m/04_sv,/m/07yv9"
22158,"zzlK8KDqlr0,","370.000,","380.000,","/m/01m2v,/m/07qc9xj,/m/09x0r,/t/dd00125"


In [ ]:
from yt_dlp import YoutubeDL
from tqdm import tqdm

DEFAULT_FS=16000
dst_dir = 'AudioSet/'
ydl_opts = {'format': 'worstaudio', 'paths': {'home': 'AudioSet/'}, 'external_downloader':'ffmpeg'}
items = df['YTID,']
items = ['https://youtu.be/'+x[:-1] for x in items]
with YoutubeDL(ydl_opts) as ydl:
  for i, url in tqdm(enumerate(items)):
    start_time = float(df['start_seconds,'][i][:-1])
    output_name = f"{dst_dir}{url.split('/')[-1]}_{start_time}.wav"
    if os.path.isfile(output_name):
      continue
    os.system(f"ffmpeg -hide_banner -loglevel error -ss {str(start_time)} -t 10 -i $(yt-dlp -f 'bestaudio' -g {url}) -ar {str(DEFAULT_FS)} -- '{output_name}'")


0it [00:00, ?it/s]ERROR: [youtube] --ZhevVpy1s: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
2it [00:01,  1.93it/s]ERROR: [youtube] --aO5cdqSAg: Video unavailable. This video is no longer available due to a privacy claim by a third party.
-ar: No such file or directory
4it [00:02,  1.92it/s]ERROR: [youtube] -0FHUc78Gqo: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
10it [00:03,  3.78it/s]ERROR: [youtube] -0Vl4HyWRk8: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
13it [00:04,  3.46it/s]ERROR: [youtube] -0mG4W5Hlq8: Video unavailable
-ar: No such file or directory
14it [00:05,  2.55it/s]ERROR: [youtube] -1TLtjPtnms: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory

360it [00:58,  8.90it/s]ERROR: [youtube] -oGhPs2S17Q: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
370it [00:59,  9.03it/s]ERROR: [youtube] -oSZw8b-vgU: This video has been removed for violating YouTube's Terms of Service.
-ar: No such file or directory
372it [01:00,  6.79it/s]ERROR: [youtube] -s9kwrRilOY: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
400it [01:01, 12.66it/s]ERROR: [youtube] -uuCV1W6jYk: Video unavailable
-ar: No such file or directory
418it [01:02, 14.09it/s]ERROR: [youtube] -uzavgbcJVA: Video unavailable
-ar: No such file or directory
420it [01:03, 10.62it/s]ERROR: [youtube] -v423vsVOk8: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
422it [01:04,  8.08it/s]ERROR: [youtube] -wMsOTnj0XE: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
430it [01:05,  8.12it/s]ERROR: 

820it [02:05, 12.15it/s]ERROR: [youtube] 0nCSWuPhU-w: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
823it [02:06,  9.52it/s]ERROR: [youtube] 0nYhDvktiJo: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
828it [02:07,  8.00it/s]ERROR: [youtube] 0o4vHYlTPBA: Video unavailable
-ar: No such file or directory
832it [02:08,  6.85it/s]ERROR: [youtube] 0ogrLUe_WSY: This video has been removed for violating YouTube's Terms of Service.
-ar: No such file or directory
836it [02:09,  5.95it/s]ERROR: [youtube] 0pr3Huz6pq8: This video has been removed for violating YouTube's policy on violent or graphic content.
-ar: No such file or directory
844it [02:10,  6.51it/s]ERROR: [youtube] 0qCbtn--t6Q: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
847it [02:11,  5.41it/s]ERROR: [youtube] 0qMF1Iby

1222it [03:15,  9.15it/s]ERROR: [youtube] 1jc3YPwYfC4: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
1225it [03:16,  7.27it/s]ERROR: [youtube] 1kfvRJEsf9g: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
1235it [03:17,  8.09it/s]ERROR: [youtube] 1lBAPeMA7-o: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
1241it [03:18,  7.40it/s]ERROR: [youtube] 1lSn0xFQLmQ: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
1244it [03:20,  4.57it/s]ERROR: [youtube] 1mg3lx0NDS0: Video unavailable
-ar: No such file or directory
1249it [03:21,  4.60it/s]ERROR: [youtube] 1nTNW_uwS7I: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
1255it 

1685it [04:19,  7.01it/s]ERROR: [youtube] 2rJn94itrBo: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
1697it [04:20,  8.52it/s]ERROR: [youtube] 2rM5bInx6wY: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
1698it [04:21,  6.18it/s]ERROR: [youtube] 2rPv0UjHwR4: Video unavailable
-ar: No such file or directory
1700it [04:22,  4.96it/s]ERROR: [youtube] 2rcRqeXnsNw: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
1701it [04:23,  3.77it/s]ERROR: [youtube] 2sCd_EwzYgY: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
1704it [04:24,  3.53it/s]ERROR: [youtube] 2sqHdyT_FUQ: Video unavailable
-ar: No such file or directory
1709it [04:25,  3.63it/s]ERROR: [youtube] 2tfiDS7q4gE: Video unavailable
-ar: No such file or directory
1712it [04:26,  3.43it/s]ERROR: [youtube] 2tgqrxUIozQ: Private video. Sign i

...

7736it [22:06,  9.51it/s]ERROR: [youtube] IaUivuWPwPw: Video unavailable
-ar: No such file or directory
7739it [22:07,  7.70it/s]ERROR: [youtube] IbnvRqjVLDM: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
7745it [22:08,  7.18it/s]ERROR: [youtube] IcoCfcyK-EI: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
7754it [22:09,  7.68it/s]ERROR: [youtube] Id39QbfhuSg: Video unavailable
-ar: No such file or directory
7756it [22:10,  6.01it/s]ERROR: [youtube] IdpY7m2hvP0: This video has been removed for violating YouTube's Terms of Service.
-ar: No such file or directory
7762it [22:11,  5.98it/s]ERROR: [youtube] IeJuqRgPGzQ: Video unavailable
-ar: No such file or directory
7764it [22:12,  4.30it/s]ERROR: [youtube] IhF37Npk30E: Video unavailable
-ar: No such file or directory
7785it [22:13,  8.77it/s]ERROR: [youtube] IhlKHgORrgM: Video

8132it [23:30,  5.61it/s]ERROR: [youtube] JimPsBLzZGw: Video unavailable
-ar: No such file or directory
8136it [23:31,  5.21it/s]ERROR: [youtube] Jlh_dAyVBf4: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
8148it [23:32,  7.10it/s]ERROR: [youtube] Jm6KScwr0NE: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
8153it [23:33,  6.51it/s]ERROR: [youtube] Jn3ubUwBTTw: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
8158it [23:34,  6.04it/s]ERROR: [youtube] JnykJB7JoKk: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
8163it [23:35,  5.67it/s]ERROR: [youtube] JpVoARn-SWU: Video unavailable
-ar: No such file or directory
8173it [23:37,  5.62it/s]ERROR: [youtube] JqQVax-LLYM: Video unavailable. This video is no longer available because th

8499it [24:32,  7.17it/s]ERROR: [youtube] KlPQRxRAVF4: Video unavailable
-ar: No such file or directory
8510it [24:33,  8.16it/s]ERROR: [youtube] KmCRDcfrp4g: Video unavailable
-ar: No such file or directory
8515it [24:34,  7.16it/s]ERROR: [youtube] KndBk6pnCxg: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
8523it [24:35,  7.38it/s]ERROR: [youtube] KorKV9fnZOQ: Video unavailable
-ar: No such file or directory
8534it [24:36,  8.24it/s]ERROR: [youtube] KpGO2VTksIw: Video unavailable
-ar: No such file or directory
8537it [24:37,  6.73it/s]ERROR: [youtube] Kq5kK9M1PiU: Video unavailable. This video contains content from The Orchard Music, who has blocked it in your country on copyright grounds.
-ar: No such file or directory
8539it [24:38,  4.99it/s]ERROR: [youtube] KqUD1AAGpJ4: Video unavailable
-ar: No such file or directory
8541it [24:39,  4.15it/s]ERROR: [youtube] Kr4Bs2F9V

8856it [25:28,  3.89it/s]ERROR: [youtube] Lf5Qv-0c0oA: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
8857it [25:29,  3.05it/s]ERROR: [youtube] LfeO5qcawrI: Private video. Sign in if you've been granted access to this video
-ar: No such file or directory
8861it [25:30,  3.31it/s]ERROR: [youtube] LfkgnaW4cqY: Video unavailable
-ar: No such file or directory
8863it [25:31,  2.88it/s]ERROR: [youtube] Lkg9SQwQYaE: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
8888it [25:32,  9.13it/s]ERROR: [youtube] Ll6CNFVFTC8: Video unavailable. This video is no longer available due to a copyright claim by SoundEffectsFactory.
-ar: No such file or directory
8891it [25:33,  7.12it/s]ERROR: [youtube] Lme-TnoW9PY: Video unavailable
-ar: No such file or directory
8896it [25:34,  6.48i

9206it [26:48,  5.42it/s]ERROR: [youtube] MmYWOUg9hgA: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
9209it [26:49,  4.84it/s]ERROR: [youtube] MnD5rqXwzww: Video unavailable
-ar: No such file or directory
9214it [26:50,  4.82it/s]ERROR: [youtube] MnbMgu6hsXg: Video unavailable
-ar: No such file or directory
9216it [26:51,  3.99it/s]ERROR: [youtube] Mnx3LMxvu50: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
9219it [26:52,  3.70it/s]ERROR: [youtube] MoXvWqQ2v38: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
-ar: No such file or directory
9225it [26:53,  4.32it/s]ERROR: [youtube] ModwjshMrQ8: Video unavailable. This video contains content from Mosfilm, who has blocked it on copyright

In [83]:
item[0][:-1]

'--PJHxphWEs'

In [ ]:
class AudioSet:
  def __init__(self, wav_dir, csv_path):
    self.dir = wav_dir
    
    df_dirty = pd.read_csv(csv_path, delimiter=' ', header=2)
    df = df_dirty.copy()
    for i in range(1, len(df.columns)):
      df[df.columns[i]] = df_dirty[df.columns[i-1]]
    self.meta = df.drop(columns='#')

  def filter_by_existence(self):
    return
  
  def __len__(self):
    return len(self.meta)
      
  def __getitem__(self, idx):
    return
    